In [1]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ И СОБЫТИЯ ЯНДЕКСА И РОССТАТ

In [2]:
events1 = pd.read_csv('события.csv')
events2 = pd.read_csv('росстат_общий_new.csv')


df = events2.merge(
    events1[['Event Text', 'region_name', 'municipality', 'year']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df.to_csv('общие_данные_2010t5.csv', index=False)
df

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256415,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN
256416,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN
256417,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
256418,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN


ОБЪЕДИНЕНИЕ ПРОТЕСТОВ С ДАННЫМИ РОССТАТА 

1. ВООБЩЕ 
2. ТОЛЬКО ЗА 2012

In [3]:
data = pd.read_csv('общие_данные_2010t5.csv')

protest_events = ["Demonstrate or rally", "Criticize or denounce", "Defy norms, law"]


protests = data[data['Event Text'].isin(protest_events)].copy()
protests_0910 = protests[protests['year'].isin([2009, 2010, 2011, 2012])].copy()

# Для каждого oktmo записываем минимальный год протеста в 2009–2010
protests_0910['fprotets_year'] = protests_0910.groupby('oktmo')['year'].transform('min')

# Оставляем по одной строке на oktmo (колонка fprotets_year уже заполнена правильно)
protests1 = protests_0910.drop_duplicates(subset=['oktmo']).reset_index(drop=True)



protests.to_csv('протесты_2010t5.csv', index=False)
protests1.to_csv('протесты2009_2013_2010t5.csv', index=False)
protests1

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text,fprotets_year
0,Абакан,95701000,79.594802,Хакасия,2009.0,2.79,0.00,400.0,860.0,0.000000,0.000000,0.00,Criticize or denounce,2009.0
1,Агрызский,92601000,25.197968,Татарстан,2010.0,29.70,81.74,0.0,0.0,0.000000,0.000000,0.00,Criticize or denounce,2010.0
2,Александровский,7602000,14.330458,Ставропольский,2010.0,0.40,0.00,0.0,0.0,0.000000,0.000000,0.00,Demonstrate or rally,2010.0
3,Александровский,7602402,0.000000,Ставропольский,2010.0,0.00,0.00,500.0,1658.5,0.000000,0.000000,0.00,Demonstrate or rally,2010.0
4,Алнашский,94602000,12.805933,Удмуртская,2012.0,12.30,1.57,182.0,1781.0,41.614192,0.000000,1887754.53,Demonstrate or rally,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Южно-Сахалинск,64701000,77.693897,Сахалинская,2011.0,123.60,96.48,200.0,700.0,25.110739,96.155110,89051666.00,Criticize or denounce,2011.0
163,Якутск,98701000,36.968829,Саха,2011.0,333.70,10.00,200.0,4500.0,58.011502,94.271863,91725493.00,Criticize or denounce,2011.0
164,Янаульский,80659000,15.041336,Башкортостан,2012.0,20.00,14.20,0.0,5800.0,43.021814,56.961782,5047203.00,Criticize or denounce,2012.0
165,Ярославль,78701000,0.000000,Ярославская,2009.0,309.50,0.00,14320.0,20870.0,0.000000,0.000000,0.00,Criticize or denounce,2009.0


ЕДИНИЦЫ БЕЗ ПРОТЕСТОВ СРЕДИ ВСЕХ СОБЫТИЙ ЗА ВСЕ ВРЕМЯ

In [4]:
lol = pd.read_csv("протесты_2010t5.csv") 
sobytia = pd.read_csv("общие_данные_2010t5.csv")


noprotestALL = sobytia[~sobytia['municipality'].isin(lol['municipality'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2010t5.csv', index=False)
noprotestALL

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175022,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN
175023,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN
175024,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
175025,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN


ПОИСК КОНТРОЛЬНОЙ ГРУППЫ (НЕПРОТЕСТНЫЕ ЕДИНИЦЫ ЗА 2011 ИЗ ТЕХ ЧТО ПРОТЕСТНЫЕ В 2012)

In [5]:
lol = pd.read_csv("протесты2009_2013_2010t5.csv") 
sobytia = pd.read_csv("общие_данные_2010t5.csv")

df_2011 = sobytia[sobytia['year'] == 2008]


filtered_df = df_2011[df_2011['oktmo'].isin(lol['oktmo'])]

pattern = "Demonstrate or rally|Criticize or denounce|Defy norms, law"
mask = filtered_df['Event Text'].str.contains(pattern, case=False, na=False)
municipalities_with_tag = filtered_df[mask]['oktmo'].unique()


final_df = filtered_df[~filtered_df['oktmo'].isin(municipalities_with_tag)].reset_index(drop=True)

final_df = final_df.drop_duplicates(subset='oktmo').reset_index(drop=True)
final_df.to_csv('без_протестов2008_2010t5.csv', index=False)
final_df



,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
0,Абакан,95701000,79.594802,Хакасия,2008.0,21.1,0.00,381.0,6300.0,0.0,0.0,0.0,"Arrest, detain, or charge with legal action"
1,Александровский,7602402,0.000000,Ставропольский,2008.0,0.0,1.46,0.0,0.0,0.0,0.0,0.0,NaN
2,Алнашский,94602000,12.805933,Удмуртская,2008.0,0.0,0.00,0.0,2535.0,0.0,0.0,0.0,Make statement
3,Анапа,3703000,88.267325,Краснодарский,2008.0,1.0,11.30,1600.0,34000.0,0.0,0.0,0.0,"Arrest, detain, or charge with legal action"
4,Архангельск,11701000,0.000000,Архангельская,2008.0,474.7,0.00,3065.0,3181.0,0.0,0.0,0.0,Make an appeal or request
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Элиста,85701000,24.905577,Калмыкия,2008.0,51.8,0.00,0.0,2700.0,0.0,0.0,0.0,Express intent to meet or negotiate
114,Якутск,98701000,36.968829,Саха,2008.0,328.5,2.16,4279.0,3192.0,0.0,0.0,0.0,"Arrest, detain, or charge with legal action"
115,Янаульский,80659000,15.041336,Башкортостан,2008.0,4.5,0.00,0.0,1500.0,0.0,0.0,0.0,NaN
116,Ярославль,78701000,0.000000,Ярославская,2008.0,325.2,0.00,16900.0,27900.0,0.0,0.0,0.0,Use unconventional violence


In [6]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)



df_protests = pd.read_csv("протесты2009_2013_2010t5.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t5.csv")
df_events_full = pd.read_csv("общие_данные_2010t5.csv")

required_years = {2008.0, 2013.0, 2014.0, 2015.0}

df_events = df_events[df_events['year'].isin([2009, 2010, 2011, 2012])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)

print("Размер df_protests:", len(df_protests))
print("Размер df_events:", len(df_events))

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()

if degenerate:
    print(f"Вырожденные признаки (дисперсия <= {threshold}): {degenerate}")
    features = [f for f in features if f not in degenerate]
    print(f"Оставшиеся признаки: {features}")
else:
    print("Нет вырожденных признаков.")

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined.T)
cov_inv = np.linalg.inv(cov_matrix)

complete_event_oktmos = []
for oktmo in df_events['oktmo'].unique():
    if check_years_completeness(oktmo, df_events_full, required_years):
        complete_event_oktmos.append(oktmo)

df_events_complete = df_events[df_events['oktmo'].isin(complete_event_oktmos)].reset_index(drop=True)

print(f"После фильтрации по годам осталось {len(df_events_complete)} муниципалитетов событий")

n_protests = df_protests.shape[0]
n_events = df_events_complete.shape[0]
distance_matrix = np.zeros((n_protests, n_events))

for i in range(n_protests):
    x = df_protests.loc[i, features].values
    for j in range(n_events):
        y = df_events_complete.loc[j, features].values
        distance_matrix[i, j] = mahalanobis(x, y, cov_inv)

row_ind, col_ind = linear_sum_assignment(distance_matrix)

df_matched_protests = df_protests.iloc[row_ind].reset_index(drop=True)
df_matched_events = df_events_complete.iloc[col_ind].reset_index(drop=True)

df_matches = df_matched_protests.join(df_matched_events, lsuffix="_protest", rsuffix="_event")
df_matches["mahalanobis_distance"] = distance_matrix[row_ind, col_ind]

df_matches.to_csv('прототип_данных_2010t5.csv', index=False)
df_matches


Размер df_protests: 157
Размер df_events: 41932
Нет вырожденных признаков.
После фильтрации по годам осталось 11786 муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2009.0,2.79,0.00,400.0,860.0,Свердловская,Невьянский,65714000,18.3,0.00,400.0,850.0,0.010289
1,Татарстан,Агрызский,92601000,2010.0,29.70,81.74,0.0,0.0,Татарстан,Верхнеуслонский,92620000,26.1,81.64,0.0,0.0,0.007167
2,Ставропольский,Александровский,7602000,2010.0,0.40,0.00,0.0,0.0,Хабаровский,Аяно-Майский,8606000,0.4,0.00,0.0,0.0,0.000000
3,Удмуртская,Алнашский,94602000,2012.0,12.30,1.57,182.0,1781.0,Татарстан,Лаишево,92634101,2.4,0.85,200.0,1500.0,0.083684
4,Башкортостан,Альшеевский,80602000,2009.0,0.80,0.00,200.0,3100.0,Мордовия,Краснослободское,89634101,0.0,0.00,200.0,3100.0,0.000514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Сахалинская,Южно-Сахалинск,64701000,2011.0,123.60,96.48,200.0,700.0,Смоленская,Кардымовский,66623000,2.9,97.24,100.0,543.0,0.121402
153,Саха,Якутск,98701000,2011.0,333.70,10.00,200.0,4500.0,Рязанская,Скопин,61715000,0.0,10.55,200.0,4600.0,0.218679
154,Башкортостан,Янаульский,80659000,2012.0,20.00,14.20,0.0,5800.0,Нижегородская,Ветлужский,22618000,10.8,15.59,0.0,5900.0,0.094941
155,Ярославская,Ярославль,78701000,2009.0,309.50,0.00,14320.0,20870.0,Новосибирская,Карасук,50617101,4.1,0.00,15480.0,28500.0,2.006289


In [7]:
df_2012 = pd.read_csv('прототип_данных_2010t5.csv')
df_2011 = pd.read_csv('без_протестов2008_2010t5.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2010t5.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,fprotets_year
0,Абакан,95701000,2.79,0.00,400.0,860.0,Невьянский,65714000,18.30,0.00,400.0,850.0,2009.0
1,Алнашский,94602000,12.30,1.57,182.0,1781.0,Лаишево,92634101,2.40,0.85,200.0,1500.0,2012.0
2,Анапа,3703000,0.90,2.22,300.0,21300.0,Володарский,12610000,46.09,0.00,100.0,21330.0,2009.0
3,Архангельск,11701000,485.50,0.00,2000.0,1300.0,Каширский,20620000,1.70,0.00,2000.0,1000.0,2009.0
4,Архаринский,10605000,47.30,0.00,0.0,700.0,Ярковский,71658000,49.00,0.00,0.0,700.0,2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Элиста,85701000,38.30,51.14,44.0,4377.0,Беловский,38602000,5.00,51.15,0.0,4400.0,2012.0
107,Якутск,98701000,333.70,10.00,200.0,4500.0,Скопин,61715000,0.00,10.55,200.0,4600.0,2011.0
108,Янаульский,80659000,20.00,14.20,0.0,5800.0,Ветлужский,22618000,10.80,15.59,0.0,5900.0,2012.0
109,Ярославль,78701000,309.50,0.00,14320.0,20870.0,Карасук,50617101,4.10,0.00,15480.0,28500.0,2009.0


Создание регрессии

In [8]:
df = pd.read_csv('общие_данные_2010t5.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010t5.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2009, 2010, 2011, 2012]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t5.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,fprotets_year
0,2008.0,Абакан,95701000,21.10,0.00,381.0,6300.0,Хакасия,Невьянский,65714000.0,22.2,0.00,500.0,200.0,Свердловская,NaN
1,2009.0,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Невьянский,65714000.0,18.0,0.00,710.0,200.0,Свердловская,2009.0
2,2010.0,Абакан,95701000,2800.00,0.00,0.0,500.0,Хакасия,Невьянский,65714000.0,17.6,0.00,400.0,2300.0,Свердловская,2009.0
3,2011.0,Абакан,95701000,2100.00,0.00,0.0,1400.0,Хакасия,Невьянский,65714000.0,18.3,0.00,400.0,850.0,Свердловская,2009.0
4,2012.0,Абакан,95701000,2.10,0.00,180.0,1946.0,Хакасия,Невьянский,65714000.0,28.2,34.76,200.0,800.0,Свердловская,2009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,2016.0,Ярославский,78650000,0.00,31.53,180.0,4336.0,Ярославская,Воткинский,94608000.0,0.0,8.50,100.0,2300.0,Удмуртская,NaN
1408,2017.0,Ярославский,78650000,0.00,32.16,360.0,4350.0,Ярославская,Воткинский,94608000.0,0.0,10.33,0.0,1000.0,Удмуртская,NaN
1409,2018.0,Ярославский,78650000,0.00,30.91,907.0,4068.0,Ярославская,Воткинский,94608000.0,0.0,11.29,50.0,1901.0,Удмуртская,NaN
1410,2019.0,Ярославский,78650000,0.00,38.30,62.0,1892.0,Ярославская,Воткинский,94608000.0,0.0,11.29,500.0,1970.0,Удмуртская,NaN


In [9]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

def matching_with_year_completeness(df_protests, df_events, df_events_full, features, required_years, cov_inv):
    matches_list = []
    
    for region in df_protests['region_name'].unique():
        sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
        sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
        
        complete_event_munis = []
        for _, row in sub_events.iterrows():
            if check_years_completeness(row['oktmo'], df_events_full, required_years):
                complete_event_munis.append(row['oktmo'])
        
        sub_events_complete = sub_events[sub_events['oktmo'].isin(complete_event_munis)].reset_index(drop=True)
        
        if len(sub_events_complete) == 0:
            print(f"В регионе {region} нет муниципалитетов событий с полными данными за все годы")
            continue
        
        n_p = sub_protests.shape[0]
        n_e = sub_events_complete.shape[0]
        dist_mat = np.zeros((n_p, n_e))
        
        for i in range(n_p):
            x = sub_protests.loc[i, features].values
            for j in range(n_e):
                y = sub_events_complete.loc[j, features].values
                dist_mat[i, j] = mahalanobis(x, y, cov_inv)
        
        row_ind, col_ind = linear_sum_assignment(dist_mat)
        
        matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
        matched_e = sub_events_complete.iloc[col_ind].reset_index(drop=True)
        matched = matched_p.join(
            matched_e,
            lsuffix="_protest",
            rsuffix="_event"
        )
        matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
        matches_list.append(matched)
    
    return matches_list

df_protests = pd.read_csv("протесты2009_2013_2010t5.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t5.csv")
df_events_full = pd.read_csv("общие_данные_2010t5.csv")

required_years = {2008.0, 2013.0, 2014.0, 2015.0}

df_events = df_events[df_events['year'].isin([2009, 2010, 2011, 2012])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo'] + features
].reset_index(drop=True)

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

matches_list = matching_with_year_completeness(
    df_protests, df_events, df_events_full, features, required_years, cov_inv
)


df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t5M.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Чеченская нет муниципалитетов событий с полными данными за все годы
В регионе Ингушетия нет муниципалитетов событий с полными данными за все годы


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2009.0,2.79,0.00,400.0,860.0,Хакасия,Черногорск,95715000,100.60,0.00,345.0,1248.0,0.125243
1,Хакасия,Таштыпский,95625000,2010.0,10830.00,0.00,0.0,10.0,Хакасия,Усть-Абаканский,95630000,9542.00,0.00,0.0,1550.0,0.915130
2,Татарстан,Агрызский,92601000,2010.0,29.70,81.74,0.0,0.0,Татарстан,Верхнеуслонский,92620000,26.10,81.64,0.0,0.0,0.007167
3,Татарстан,Казань,92701000,2010.0,419.40,0.00,1100.0,12660.0,Татарстан,Лаишево,92634101,0.00,5.24,0.0,3700.0,2.379084
4,Ставропольский,Александровский,7602000,2010.0,0.40,0.00,0.0,0.0,Ставропольский,Высоцкий,7646404,0.46,0.00,0.0,0.0,0.000039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Карачаево-Черкесская,Усть-Джегутинский,91635000,2010.0,19.00,0.00,0.0,3100.0,Карачаево-Черкесская,Усть-Джегутинское,91635101,3.00,0.00,0.0,3097.0,0.010310
149,Ростовская,Целинский,60656000,2009.0,12.10,0.00,0.0,2000.0,Ростовская,Пролетарский,60645000,10.00,0.00,0.0,2100.0,0.025223
150,Чувашская,Чебоксарский,97644000,2012.0,11.20,41.61,0.0,5200.0,Чувашская,Ядринский,97653000,4.00,40.76,100.0,3800.0,0.364756
151,Калмыкия,Элиста,85701000,2012.0,38.30,51.14,44.0,4377.0,Калмыкия,Соленовское,85650466,0.00,0.00,0.0,1500.0,3.460457


In [10]:
df_2012 = pd.read_csv('прототип_данных_2010t5M.csv')
df_2011 = pd.read_csv('без_протестов2008_2010t5.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2010t5M.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,fprotets_year
0,Абакан,95701000,2.79,0.00,400.0,860.0,Черногорск,95715000,100.60,0.00,345.0,1248.0,2009.0
1,Таштыпский,95625000,10830.00,0.00,0.0,10.0,Усть-Абаканский,95630000,9542.00,0.00,0.0,1550.0,2010.0
2,Казань,92701000,419.40,0.00,1100.0,12660.0,Лаишево,92634101,0.00,5.24,0.0,3700.0,2010.0
3,Лермонтов,7718000,0.00,0.00,228.0,242.0,Краснооктябрьский,7612409,0.00,0.00,200.0,150.0,2012.0
4,Невинномысск,7724000,11.00,79.58,0.0,925.0,Изобильненский,7620000,2.90,81.73,0.0,0.0,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Усть-Джегутинский,91635000,19.00,0.00,0.0,3100.0,Усть-Джегутинское,91635101,3.00,0.00,0.0,3097.0,2010.0
105,Целинский,60656000,12.10,0.00,0.0,2000.0,Пролетарский,60645000,10.00,0.00,0.0,2100.0,2009.0
106,Чебоксарский,97644000,11.20,41.61,0.0,5200.0,Ядринский,97653000,4.00,40.76,100.0,3800.0,2012.0
107,Элиста,85701000,38.30,51.14,44.0,4377.0,Соленовское,85650466,0.00,0.00,0.0,1500.0,2012.0


In [11]:
df = pd.read_csv('общие_данные_2010t5.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010t5M.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2009, 2010, 2011, 2012]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t5M.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,fprotets_year
0,2008.0,Абакан,95701000,21.10,0.00,381.0,6300.0,Хакасия,Черногорск,95715000.0,104.3,0.00,227.0,1639.0,Хакасия,NaN
1,2009.0,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Черногорск,95715000.0,100.6,0.00,345.0,1248.0,Хакасия,2009.0
2,2010.0,Абакан,95701000,2800.00,0.00,0.0,500.0,Хакасия,Черногорск,95715000.0,100500.0,0.00,306.0,2402.0,Хакасия,2009.0
3,2011.0,Абакан,95701000,2100.00,0.00,0.0,1400.0,Хакасия,Черногорск,95715000.0,105400.0,0.00,346.0,2402.0,Хакасия,2009.0
4,2012.0,Абакан,95701000,2.10,0.00,180.0,1946.0,Хакасия,Черногорск,95715000.0,104.2,0.00,374.0,2101.0,Хакасия,2009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,2016.0,Якутск,98701000,0.00,25.00,1900.0,1600.0,Саха,Мирнинский,98631000.0,0.0,63.41,2020.0,9370.0,Саха,NaN
1382,2017.0,Якутск,98701000,0.00,25.00,0.0,1100.0,Саха,Мирнинский,98631000.0,0.0,64.37,670.0,5410.0,Саха,NaN
1383,2018.0,Якутск,98701000,0.00,25.00,0.0,3510.0,Саха,Мирнинский,98631000.0,0.0,60.68,680.0,5830.0,Саха,NaN
1384,2019.0,Якутск,98701000,0.00,29.05,1320.0,2830.0,Саха,Мирнинский,98631000.0,0.0,47.74,520.0,5840.0,Саха,NaN


In [12]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

df_protests = pd.read_csv("протесты2009_2013_2010t5.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t5.csv")
df_events_full = pd.read_csv("общие_данные_2010t5.csv")

required_years = {2008.0, 2013.0, 2014.0, 2015.0}


df_events = df_events[df_events['year'].isin([2009, 2010, 2011, 2012])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)

exclude_dict = {
        'Абакан': ['Усть-Абаканский', 'Алтайский', 'Минусинский', 'Минусинск', 'Черногорск'],
    'Архангельск': ['Северодвинск', 'Приморский'],
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Вологда': ['Вологодский'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Ярославль': ['Ярославский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский', 'Новоильмовское'],
    'Омск': ['Омский'],
    'Оренбург': ['Оренбургский', 'Сакмарский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Иркутск': ['Иркутский', 'Ангарское', 'Ангарск', 'Шелеховский'],
    'Калуга': ['Перемышльский', 'Ферзиковский', 'Малоярославецкий', 'Дзержинский', 'Бабынинский'],
    'Анапа': ['Темрюкский', 'Крымский', 'Новороссийск'],
    'Балашиха': ['Королев', 'Люберцы', 'Богородский', 'Щелково', 'Раменский'],
    'Бокситогорский': ['Тихвинский', 'Любытинский', 'Хвойнинский', 'Чагодощенский', 'Бабаевский'],
    'Варгашинский': ['Кетовский', 'Курган', 'Половинский', 'Лебяжьевский', 'Мокроусовский', 'Упоровский', 'Белозерский'],
    'Горно-Алтайск': ['Майминский'],
    'Добрянский': ['Краснокамский', 'Ильинский', 'Юсьвинский', 'Березники', 'Александровский', 'Губаха', 'Чусовской', 'Пермский'],
    'Кола': ['Североморск', 'Александровск', 'Кольский', 'Кильдинстрой'],
    'Комсомольск-на-Амуре': ['Комсомольский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Красноярский': ['Володарский', 'Приволжский', 'Наримановский', 'Харабалинский'],
    'Курганинский': ['Лабинский', 'Новокубанский', 'Гулькевичский', 'Тбилисский', 'Шовгеновский', 'Кошехабльский'],
    'Новороссийск': ['Новороссийский'],
    'Орёл': ['Орловский'],
    'Почепский': ['Трубчевский', 'Погарский', 'Унечский', 'Мглинский', 'Клетнянский', 'Жирятинский', 'Выгоничский', 'Титовское'],
    'Ростов': ['Ростовский'],
    'Саратовский': ['Красноармейский', 'Саратов', 'Лысогорский', 'Татищевский'],
    'Тольятти': ['Жигулевск', 'Ставропольский'],
    'Учалинский': ['Белорецкий', 'Катав-Ивановский', 'Верхнеуральский', 'Уйский', 'Чебаркульский', 'Миасский', 'Златоустовский'],
    'Целинский': ['Егорлыкский', 'Зерноградский', 'Сальский', 'Песчанокопский', 'Белоглинский'],
    'Абазинский': ['Хабезский', 'Усть-Джегутинский', 'Черкесский', 'Прикубанский', 'Адыге-Хабльский'],
    'Алнашский': ['Граховский', 'Менделеевский', 'Агрызский', 'Можгинский'],
    'Апатиты': ['Полярные Зори', 'Кандалакшский', 'Терский', 'Кировск', 'Оленегорск', 'Мончегорск'],
    'Архаринский': ['Облученский', 'Верхнебуреинский', 'Бурейский'],
    'Асекеевский': ['Грачёвский', 'Матвеевский', 'Абдулинский', 'Бугурусланский', 'Бузулукский'],
    'Аургазинский': ['Кармаскалинский', 'Давлекановский', 'Альшеевский', 'Стерлитамакский', 'Гафурийский'],
    'Бабаюрт': ['Шелковской', 'Хасавюртовский', 'Кизилюртовский', 'Кумторкалинский', 'Кизлярский'],
    'Баксан': ['Баксанский', 'Чегемский', 'Прохладненский'],
    'Баксанский': ['Баксан', 'Чегемский', 'Прохладненский', 'Зольский', 'Эльбрусский', 'Кировский'],
    'Бердск': ['Новосибирский', 'Искитим', 'Искитимский', 'Кольцово'],
    'Благовещенск': ['Благовещенский', 'Кушнаревский', 'Имангуловский'],
    'Буйнакск': ['Буйнакский'],
    'Воловский': ['Тепло-Огаревский', 'Каменский', 'Ефремов', 'Куркинский', 'Богородицкий'],
    'Волосовский': ['Кингисеппский', 'Сланцевский', 'Лужский', 'Гатчинский', 'Ломоносовский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Гатчинский': ['Лужский', 'Волосовский', 'Тосненский', 'Пушкин', 'Ломоносовский'],
    'Геленджик': ['Туапсинский', 'Северский', 'Абинский', 'Крымский', 'Новороссийск'],
    'Грязовецкий': ['Вологодский', 'Междуреченский', 'Солигаличский', 'Буйский', 'Любимский', 'Первомайский', 'Пошехонский'],
    'Гулькевичский': ['Кавказский', 'Новоалександровский', 'Новокубанский', 'Курганинский', 'Тбилисский'],
    'Гурьевский': ['Зеленоградский', 'Калининград', 'Полесский', 'Гвардейский', 'Багратионовский', 'Ладушкинский'],
    'Даниловский': ['Котовский', 'Руднянский', 'Еланский', 'Михайловка', 'Фроловский', 'Ольховский'],
    'Дербент': ['Дербентский', 'Докузпаринский', 'Курахский', 'Дагестанские Огни'],
    'Дмитриевский': ['Хомутовский', 'Севский', 'Комаричский', 'Дмитровский', 'Железногорский', 'Конышевский'],
    'Домбаровский': ['Ясненский', 'Комаровский', 'Новоорский', 'Орск'],
    'Домодедово': ['Подольск', 'Ленинский', 'Раменский', 'Чехов', 'Ступино', 'Щербинка'],
    'Дубна': ['Кимрский', 'Кимры', 'Талдомский', 'Дмитровский', 'Конаковский'],
    'Жуковский': ['Раменский', 'Лыткарино', 'Люберцы', 'Бронницы'],
    'Завьяловский': ['Ижевск', 'Увинский', 'Малопургинский', 'Сарапульский', 'Чайковский', 'Воткинский', 'Якшур-Бодьинский'],
    'Залегощенский': ['Орёл', 'Орловский', 'Свердловский', 'Покровский', 'Верховский', 'Новосильский', 'Мценский'],
    'Заполярный': ['Мезенский', 'Усть-Цилемский', 'Усинск', 'Инта', 'Воркута'],
    'Ивановский': ['Иваново', 'Фурмановский', 'Комсомольский', 'Тейковский', 'Лежневский', 'Шуйский', 'Родниковский'],
    'Избербаш': ['Каякентский', 'Сергокалинский', 'Дахадаевский', 'Карабудахкентский'],
    'Кашарский': ['Миллеровский', 'Чертковский', 'Верхнедонской', 'Боковский', 'Советский', 'Милютинский', 'Тарасовский'],
    'Кизильский': ['Карталинский', 'Брединский', 'Кваркенский', 'Баймакский', 'Сибай', 'Абзелиловский', 'Агаповский'],
    'Кизляр': ['Шелковской', 'Кизлярский', 'Тарумовский', 'Бабаюртовский', 'Цунтинский', 'Тляратинский', 'Гумбетовский'],
    'Кингисеппский': ['Сланцевский', 'Волосовский', 'Ломоносовский', 'Сосновоборский'],
    'Кинель': ['Волжский', 'Кинельский'],
    'Кинельский': ['Кинель', 'Волжский', 'Красноярский', 'Кинель-Черкасский', 'Богатовский', 'Нефтегорский', 'Приволжье'],
    'Кировский': ['Куйбышевский', 'Дятьковский', 'Людиновский', 'Думиничский', 'Сухиничский', 'Барятинский', 'Спас-Деменский'],
    'Кисловодск': ['Предгорный', 'Малокарачаевский', 'Зольский'],
    'Комаричский': ['Суземский', 'Севский', 'Дмитриевский', 'Дмитровский', 'Брасовский'],
    'Крестецкий': ['Новгородский', 'Парфинский', 'Демянский', 'Валдайский', 'Окуловский', 'Маловишерский'],
    'Курумканский': ['Северо-Байкальский', 'Баунтовский', 'Баргузинский'],
    'Ленинский': ['Домодедово', 'Подольск', 'Раменский', 'Лыткарино', 'Дзержинский', 'Подольск'],
    'Лермонтов': ['Предгорный', 'Железноводск', 'Пятигорск', 'Ессентуки', 'Минераловодский'],
    'Ловозерский': ['Терский', 'Кировск', 'Оленегорск', 'Кольский'],
    'Лыткарино': ['Люберцы', 'Раменский', 'Ленинский', 'Дзержинский'],
    'Магадан': ['Ольский', 'Хасынский'],
    'Макаровский': ['Долинский', 'Томаринский', 'Углегорский', 'Поронайский', 'Вахрушев'],
    'Мегион': ['Нижневартовский', 'Нижневартовск', 'Лангепас'],
    'Мезенский': ['Заполярный', 'Усть-Цилемский', 'Лешуконский', 'Пинежский', 'Приморский', 'Урдомское'],
    'Михайловский': ['Новомосковск', 'Кимовский', 'Скопинский', 'Пронский', 'Захаровский', 'Серебряные Пруды'],
    'Надымский': ['Пуровский', 'Сургутский', 'Белоярский', 'Приуральский', 'Ямальский', 'Тазовский', 'Новый Уренгой'],
    'Невинномысск': ['Кочубеевский'],
    'Некоузский': ['Мышкинский', 'Сонковский', 'Краснохолмский', 'Брейтовский', 'Рыбинский'],
    'Новгородский': ['Старорусский', 'Парфинский', 'Крестецкий', 'Маловишерский', 'Чудовский', 'Тосненский', 'Лужский', 'Батецкий', 'Шимский'],
    'Нягань': ['Октябрьский'],
    'Одинцовский': ['Власиха', 'Внуковское', 'Кокошкинское', 'Можайский', 'Красногорск', 'Марушкинское', 'Наро-Фоминский', 'Рузский', 'Истра'],
    'Палласовский': ['Быковский', 'Николаевский', 'Старополтавский', 'Знаменск', 'Жанибекский'],
    'Пинежский': ['Лешуконский', 'Мезенский', 'Приморский', 'Холмогорский', 'Виноградовский', 'Верхнетоемский', 'Удорский'],
    'Прибайкальский': ['Кабанский', 'Иволгинский', 'Улан-Удэ', 'Заиграевский', 'Хоринский', 'Баргузинский', 'Усть-Джилиндинское'],
    'Пушкинский': ['Мытищи', 'Дмитровский', 'Сергиево-Посадский', 'Красноармейск', 'Щёлково', 'Восточный', 'Ивантеевка'],
    'Салехард': ['Приуральский', 'Шурышкарский'],
    'Северо-Курильский': ['Курильский', 'Елизовский', 'Усть-Большерецкий'],
    'Северо-Эвенский': ['Пенжинский', 'Карагинский', 'Билибинский', 'Среднеканский', 'Омсукчанский'],
    'Северодвинск': ['Приморский', 'Архангельск'],
    'Северский': ['Туапсинский', 'Геленджик', 'Абинский', 'Горячий Ключ', 'Тахтамукайский', 'Красноармейский'],
    'Серафимовичский': ['Советский', 'Боковский', 'Шолоховский', 'Кумылженский', 'Михайловка', 'Фроловский', 'Клетский'],
    'Сергиевский': ['Красноярский', 'Елховский', 'Кошкинский', 'Челно-Вершинский', 'Шенталинский', 'Похвистневский', 'Кинель-Черкасский'],
    'Слюдянский': ['Закаменский', 'Тункинский', 'Кабанский', 'Иркутский', 'Шелеховский', 'Усольский', 'Баяндаевский'],
    'Солигаличский': ['Буйский', 'Галичский', 'Чухломский', 'Бабушкинский', 'Тотемский', 'Междуреченский', 'Грязовецкий'],
    'Сыктывдинский': ['Сыктывкар', 'Сысольский', 'Ленский', 'Усть-Вымский', 'Княжпогостский', 'Корткеросский', 'Койгородский'],
    'Таштыпский': ['Абаза', 'Таштагольский', 'Турочакский', 'Улаганский', 'Бай-Тайгинский', 'Барун-Хемчикский', 'Сут-Хольский', 'Шушенский', 'Бейский', 'Аскизский'],
    'Тимашевский': ['Кореновский', 'Динский', 'Калининский', 'Приморско-Ахтарский', 'Брюховецкий'],
    'Тобольск': ['Тобольский'],
    'Торжокский': ['Кувшиновский', 'Старицкий', 'Калининский', 'Лихославльский', 'Спировский', 'Вышневолоцкий', 'Шепелевское'],
    'Торопецкий': ['Западнодвинский', 'Андреапольский', 'Холмский', 'Локнянский', 'Великолукский', 'Куньинский'],
    'Тотемский': ['Бабушкинский', 'Чухломский', 'Солигаличский', 'Междуреченский', 'Сокольский', 'Сямженский', 'Верховажский', 'Тарногский', 'Нюксенский'],
    'Тюмень': ['Тюменский', 'Нижнетавдинский'],
    'Хасавюрт': ['Хасавюртовский', 'Новолакский', 'Ахвахский'],
    'Хвойнинский': ['Боровичский', 'Мошенской', 'Пестовский', 'Чагодощенский', 'Бокситогорский', 'Любытинский'],
    'Холмогорский': ['Приморский', 'Пинежский', 'Виноградовский', 'Мирный', 'Плесецкий'],
    'Чердынский': ['Троицко-Печорский', 'Красновишерский', 'Соликамский', 'Косинский', 'Гайнский', 'Усть-Куломский'],
    'Череповец': ['Череповецкий'],
    'Чернский': ['Мценский', 'Корсаковский', 'Каменский', 'Тепло-Огаревский', 'Плавский', 'Арсеньевский', 'Славный'],
    'Шуйский': ['Шуя', 'Савинский', 'Южский', 'Палехский', 'Родниковский', 'Ивановский', 'Лежневский'],
    'Янаульский': ['Куединский', 'Татышлинский', 'Бураевский', 'Калтасинский', 'Краснокамский', 'Камбарский', 'Нефтекамск', 'Чайковский'],
    'Ярославский': ['Ярославль', 'Тутаевский', 'Большесельский', 'Борисоглебский', 'Гаврилов-Ямский', 'Некрасовский', 'Даниловский'],
    'Архангельский':       ['Гафурийский', 'Белорецкий', 'Ашинский', 'Иглинский', 'Кармаскалинский'],
    'Березники':           ['Соликамский', 'Александровский', 'Добрянский', 'Юсьвинский', 'Кудымкарский', 'Косинский'],
    'Воловский':           ['Богородицкий', 'Куркинский', 'Ефремов', 'Каменский', 'Тепло-Огаревский'],
    'Комсомольский':       ['Нефтегорский', 'Богатовский', 'Кинель-Черкасский', 'Красноярский', 'Кинель', 'Волжский'],
    'Петропавловск‑Камчатский': ['Вилючинский', 'Елизовский'],
    'Усть-Джегутинский':   ['Усть-Джегутинское'],
    'Пятигорск':           ['Предгорный', 'Ессентуки', 'Лермонтов', 'Железноводск'],
    'Саранск':             ['Рузаевский', 'Лямбирский', 'Кочкуровский'],
    'Шаховская':           ['Можайский', 'Гагаринский', 'Зубцовский', 'Лотошинский', 'Волоколамский'],
    'Чебоксарский':        ['Чебоксары', 'Моргаушский', 'Красноармейский', 'Цивильский', 'Мариинско-Посадский', 'Звениговский', 'Килемарский'],
    'Чегемский':           ['Нальчик', 'Черекский', 'Эльбрусский', 'Баксанский', 'Баксан', 'Урванский'],
    'Элиста':              ['Ики-Бурульский', 'Приютненский', 'Целинный'],
    'Якутск':              ['Хангаласский', 'Мегино-Кангаласский', 'Горный', 'Намский', 'Усть-Алданский'],
    "Александровский": [
        "Петровский", "Благодарненский", "Новоселицкий",
        "Георгиевский", "Минераловодский", "Андроповский", "Грачёвский"
    ]
}

excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с усиленной проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t5M2.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Чеченская нет подходящих муниципалитетов событий
В регионе Ингушетия нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2009.0,2.79,0.00,400.0,860.0,Хакасия,Орджоникидзевский,95620000,30.90,0.00,0.0,800.0,0.272025
1,Хакасия,Таштыпский,95625000,2010.0,10830.00,0.00,0.0,10.0,Хакасия,Саяногорск,95708000,6000.00,0.00,0.0,0.0,3.104641
2,Татарстан,Агрызский,92601000,2010.0,29.70,81.74,0.0,0.0,Татарстан,Азнакаевский,92602000,22.60,81.58,0.0,0.0,0.011774
3,Татарстан,Казань,92701000,2010.0,419.40,0.00,1100.0,12660.0,Татарстан,Лаишево,92634101,0.00,5.24,0.0,3700.0,2.379084
4,Ставропольский,Александровский,7602000,2010.0,0.40,0.00,0.0,0.0,Ставропольский,Высоцкий,7646404,0.46,0.00,0.0,0.0,0.000039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Карачаево-Черкесская,Усть-Джегутинский,91635000,2010.0,19.00,0.00,0.0,3100.0,Карачаево-Черкесская,Преградненское,91630425,0.00,0.00,0.0,3020.0,0.023533
149,Ростовская,Целинский,60656000,2009.0,12.10,0.00,0.0,2000.0,Ростовская,Пролетарский,60645000,10.00,0.00,0.0,2100.0,0.025223
150,Чувашская,Чебоксарский,97644000,2012.0,11.20,41.61,0.0,5200.0,Чувашская,Ядринский,97653000,4.00,40.76,100.0,3800.0,0.364756
151,Калмыкия,Элиста,85701000,2012.0,38.30,51.14,44.0,4377.0,Калмыкия,Соленовское,85650466,0.00,0.00,0.0,1500.0,3.460457


In [13]:
df_2012 = pd.read_csv('прототип_данных_2010t5M2.csv')
df_2011 = pd.read_csv('без_протестов2008_2010t5.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)


merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2010t5M2.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,fprotets_year
0,Абакан,95701000,2.79,0.00,400.0,860.0,Орджоникидзевский,95620000,30.90,0.00,0.0,800.0,2009.0
1,Таштыпский,95625000,10830.00,0.00,0.0,10.0,Саяногорск,95708000,6000.00,0.00,0.0,0.0,2010.0
2,Казань,92701000,419.40,0.00,1100.0,12660.0,Лаишево,92634101,0.00,5.24,0.0,3700.0,2010.0
3,Лермонтов,7718000,0.00,0.00,228.0,242.0,Краснооктябрьский,7612409,0.00,0.00,200.0,150.0,2012.0
4,Невинномысск,7724000,11.00,79.58,0.0,925.0,Изобильненский,7620000,2.90,81.73,0.0,0.0,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Усть-Джегутинский,91635000,19.00,0.00,0.0,3100.0,Преградненское,91630425,0.00,0.00,0.0,3020.0,2010.0
105,Целинский,60656000,12.10,0.00,0.0,2000.0,Пролетарский,60645000,10.00,0.00,0.0,2100.0,2009.0
106,Чебоксарский,97644000,11.20,41.61,0.0,5200.0,Ядринский,97653000,4.00,40.76,100.0,3800.0,2012.0
107,Элиста,85701000,38.30,51.14,44.0,4377.0,Соленовское,85650466,0.00,0.00,0.0,1500.0,2012.0


In [14]:
df = pd.read_csv('общие_данные_2010t5.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010t5M2.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2009, 2010, 2011, 2012]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t5M2.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,fprotets_year
0,2008.0,Абакан,95701000,21.10,0.00,381.0,6300.0,Хакасия,Орджоникидзевский,95620000.0,17.10,0.00,0.0,0.0,Хакасия,NaN
1,2009.0,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Орджоникидзевский,95620000.0,30.90,0.00,0.0,800.0,Хакасия,2009.0
2,2010.0,Абакан,95701000,2800.00,0.00,0.0,500.0,Хакасия,Орджоникидзевский,95620000.0,34717.00,0.00,0.0,1600.0,Хакасия,2009.0
3,2011.0,Абакан,95701000,2100.00,0.00,0.0,1400.0,Хакасия,Орджоникидзевский,95620000.0,34717.00,0.00,0.0,500.0,Хакасия,2009.0
4,2012.0,Абакан,95701000,2.10,0.00,180.0,1946.0,Хакасия,Орджоникидзевский,95620000.0,35.68,0.00,0.0,600.0,Хакасия,2009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,2016.0,Якутск,98701000,0.00,25.00,1900.0,1600.0,Саха,Мирнинский,98631000.0,0.00,63.41,2020.0,9370.0,Саха,NaN
1382,2017.0,Якутск,98701000,0.00,25.00,0.0,1100.0,Саха,Мирнинский,98631000.0,0.00,64.37,670.0,5410.0,Саха,NaN
1383,2018.0,Якутск,98701000,0.00,25.00,0.0,3510.0,Саха,Мирнинский,98631000.0,0.00,60.68,680.0,5830.0,Саха,NaN
1384,2019.0,Якутск,98701000,0.00,29.05,1320.0,2830.0,Саха,Мирнинский,98631000.0,0.00,47.74,520.0,5840.0,Саха,NaN
